# Import

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, accuracy_score, classification_report, confusion_matrix


# Download

In [7]:
data_train= pd.read_csv(r'C:\Users\loulo\OneDrive\Bureau\cours\INF8245E\competi\train.csv', na_values=['Insert Band Name', 'Insert Band Genre', 'Insert Band Country of Origin'])
data_test = pd.read_csv(r'C:\Users\loulo\OneDrive\Bureau\cours\INF8245E\competi\test.csv', na_values=['Insert Band Name', 'Insert Band Genre', 'Insert Band Country of Origin'])
X_train = data_train.drop(columns=['Concert Enjoyment', 'Id'])
y_train = data_train['Concert Enjoyment']
X_test = data_test.drop(columns=['Id'])

# Pre processing

## encodage

In [37]:
enc = LabelEncoder()
enc.fit(X_train['Concert Goer Country of Origin'])
X_train['Band Country of Origin'],X_train['Concert Goer Country of Origin'] = enc.transform(X_train['Band Country of Origin']),enc.transform(X_train['Concert Goer Country of Origin'])
X_test['Band Country of Origin'], X_test['Concert Goer Country of Origin'] = enc.transform(X_test['Band Country of Origin']),enc.transform(X_test['Concert Goer Country of Origin'])
enc.fit(X_train['Band Name'])
X_train['Band Name'],X_test['Band Name'] = enc.transform(X_train['Band Name']),enc.transform(X_test['Band Name'])
enc.fit(X_train['Band Genre'])
X_train['Band Genre'],X_test['Band Genre']  = enc.transform(X_train['Band Genre']),enc.transform(X_test['Band Genre'])
enc.fit(X_train['Concert Goer ID'])
X_train['Concert Goer ID'],X_test['Concert Goer ID'] = enc.transform(X_train['Concert Goer ID']),enc.transform(X_test['Concert Goer ID'])

## Colonne 

On vient remplir les valeurs Nan des colonnes dans le dataset d'entrainement et de test quand cela est possible. 

In [38]:
new_df = pd.concat([X_train,X_test])
colonnes = new_df.columns

#### Colonne 0, 1 et 2 : Band Name, Band Genre et Band country of Origin 

In [39]:
results = list(new_df[colonnes[0]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[0], 'on a', somme, 'entrées')

results = list(new_df[colonnes[1]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[1], 'on a', somme, 'entrées')
results = list(new_df[colonnes[2]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[2], 'on a', somme, 'entrées')


pour Band Name on a 200000 entrées
pour Band Genre on a 200000 entrées
pour Band Country of Origin on a 200000 entrées


#### Colonne 3 : Band Debut

Il manque certaines années de début pour certains groupes. On peut les retrouver en prenant la valeur indiquées pour le même groupe et d'autres concerts 

In [40]:
results = list(new_df[colonnes[3]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[3], 'on a', somme, 'entrées')

pour Band Debut on a 198986 entrées


In [41]:
#2 secondes
for l in range (21):
    liste_des_bandes_sans_debut = new_df[new_df['Band Debut'].isna()]['Band Name'].unique()
    liste_des_bandes_sans_debut_index = new_df[new_df['Band Debut'].isna()].index
    for k in range (len(liste_des_bandes_sans_debut)): 
        val = (new_df[new_df['Band Name']==liste_des_bandes_sans_debut[k]]['Band Debut'].value_counts()).index[0]
        new_df.loc[liste_des_bandes_sans_debut_index[k], 'Band Debut'] = val

In [42]:
results = list(new_df[colonnes[3]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[3], 'on a maitenant', somme, 'entrées')

pour Band Debut on a maitenant 200000 entrées


#### Colonne 4 : Concert ID

Il manque certains concert id. On remplit les manquants en considérant que c'est le même ID que d'autres concerts 
- du même groupe 
- du même genre 
- même nombre de spectateurs
- même condition de pluie et venue en intérieur
- différent goer id

In [43]:
results = list(new_df[colonnes[4]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[4], 'on a', somme, 'entrées')

pour Concert ID on a 198972 entrées


In [44]:
#31 secondes
list_csi_index = new_df[new_df['Concert ID'].isna()].index
for k in range (len(list_csi_index)):
    if len((new_df[(new_df['Band Name'] == new_df.iloc[list_csi_index[k]]['Band Name']) & (new_df['Band Genre'] == new_df.iloc[list_csi_index[k]]['Band Genre']) & (new_df['Inside Venue'] == new_df.iloc[list_csi_index[k]]['Inside Venue']) & (new_df['Rain'] == new_df.iloc[list_csi_index[k]]['Rain']) & (new_df['Concert Attendance'] == new_df.iloc[list_csi_index[k]]['Concert Attendance']) & (new_df['Concert Goer ID'] != new_df.iloc[list_csi_index[k]]['Concert Goer ID'])])) > 0 : 
        val = (new_df[(new_df['Band Name'] == new_df.iloc[list_csi_index[k]]['Band Name']) & (new_df['Band Genre'] == new_df.iloc[list_csi_index[k]]['Band Genre']) & (new_df['Inside Venue'] == new_df.iloc[list_csi_index[k]]['Inside Venue']) & (new_df['Rain'] == new_df.iloc[list_csi_index[k]]['Rain']) & (new_df['Concert Attendance'] == new_df.iloc[list_csi_index[k]]['Concert Attendance']) & (new_df['Concert Goer ID'] != new_df.iloc[list_csi_index[k]]['Concert Goer ID'])]['Concert ID'].value_counts()).index[0]
        new_df.loc[list_csi_index[k], 'Concert ID']=val

In [45]:
results = list(new_df[colonnes[4]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[4], 'on a maintenant', somme, 'entrées')

pour Concert ID on a maintenant 199977 entrées


Il reste 23 entrées sans concert ID, toute dans le train set. On peut décider de retirer ses lignes ou non. Voici la commande pour les retirer

In [46]:

#new_df = new_df[~new_df['Concert ID'].isna()]

#### Colonne 5 : Concert Attendance

In [47]:
results = list(new_df[colonnes[5]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[5], 'on a', somme, 'entrées')

pour Concert Attendance on a 198953 entrées


In [48]:
liste_csa = list(new_df[new_df['Concert Attendance'].isna()]['Concert ID'])
liste_csa_index = new_df[new_df['Concert Attendance'].isna()].index
for k in range (len(liste_csa)):
    if len(new_df[new_df['Concert ID']==liste_csa[k]]['Concert Attendance']) >0 :
        val = (new_df[new_df['Concert ID']==liste_csa[k]]['Concert Attendance'].value_counts()).index[0]
        new_df.loc[liste_csa_index[k], 'Concert Attendance'] = val

In [49]:
results = list(new_df[colonnes[5]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[5], 'on a maintenant', somme, 'entrées')

pour Concert Attendance on a maintenant 199995 entrées


In [50]:
for name in ['Concert ID',	'Concert Attendance']:
  new_df[name].fillna(-1, inplace = True)

#### Colonne 6 : Inside Venue 

In [51]:
results = list(new_df[colonnes[6]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[6], 'on a', somme, 'entrées')

pour Inside Venue on a 198996 entrées


In [52]:
liste_iv = list(new_df[new_df['Inside Venue'].isna()]['Concert ID'])
liste_iv_index = new_df[new_df['Inside Venue'].isna()].index
for k in range (len(liste_iv)):
    if len(new_df[new_df['Concert ID']==liste_iv[k]]['Inside Venue'])>0:
        val = (new_df[new_df['Concert ID']==liste_iv[k]]['Inside Venue'].value_counts()).index[0]
        new_df.loc[liste_iv_index[k], 'Inside Venue'] = val

In [53]:
results = list(new_df[colonnes[6]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[6], 'on a', somme, 'entrées')

pour Inside Venue on a 200000 entrées


#### Colonne 7 : Rain

In [54]:
results = list(new_df[colonnes[7]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[7], 'on a', somme, 'entrées')

pour Rain on a 198996 entrées


In [55]:
liste_r = list(new_df[new_df['Rain'].isna()]['Concert ID'])
liste_r_index = new_df[new_df['Rain'].isna()].index
for k in range (len(liste_r)):
    val = (new_df[new_df['Concert ID']==liste_r[k]]['Rain'].value_counts()).index[0]
    new_df.loc[liste_r_index[k], 'Rain'] = val

In [56]:
results = list(new_df[colonnes[7]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[7], 'on a maintenant' , somme, 'entrées')

pour Rain on a maintenant 200000 entrées


#### Colonne 8 : Seated
Au sein d'un même concert les spectateurs ne sont pas tous assis, on ne peut aps deviner cette information quand elle est manquante

In [57]:
for name in ['Seated']:
  new_df[name].fillna(0, inplace = True)

#### Colonne 9,10,11,12 : Personnality traits

In [58]:
Lscp = list(new_df[new_df['Personnality Trait 1'].isna()].index)
for m in range (len(Lscp)): 
    inter_1 = new_df.iloc[Lscp[m]]['Concert Goer ID']
    inter_2 = (new_df[new_df['Concert Goer ID'] == inter_1]['Personnality Trait 1'].value_counts()).index[0]
    new_df.loc[Lscp[m], 'Personnality Trait 1']= inter_2

Lscp2 = list(new_df[new_df['Personnality Trait 2'].isna()].index)
for n in range (len(Lscp2)): 
    inter_3 = new_df.iloc[Lscp2[n]]['Concert Goer ID']
    inter_4 = (new_df[new_df['Concert Goer ID'] == inter_3]['Personnality Trait 2'].value_counts()).index[0]
    new_df.loc[Lscp2[n], 'Personnality Trait 2']= inter_4

Lscp3 = list(new_df[new_df['Personnality Trait 3'].isna()].index)
for o in range (len(Lscp3)): 
    inter_5 = new_df.iloc[Lscp3[o]]['Concert Goer ID']
    inter_6 = (new_df[new_df['Concert Goer ID'] == inter_5]['Personnality Trait 3'].value_counts()).index[0]
    new_df.loc[Lscp3[o], 'Personnality Trait 3']= inter_6

Lscp4 = list(new_df[new_df['Personnality Trait 4'].isna()].index)
for p in range (len(Lscp4)): 
    inter_7 = new_df.iloc[Lscp4[p]]['Concert Goer ID']
    inter_8 = (new_df[new_df['Concert Goer ID'] == inter_7]['Personnality Trait 4'].value_counts()).index[0]
    new_df.loc[Lscp4[p], 'Personnality Trait 4']= inter_8

#### Colonne 13 : Concert Goer Age 

In [59]:
results = list(new_df[colonnes[13]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[13], 'on a' , somme, 'entrées')

pour Concert Goer Age on a 199021 entrées


In [60]:
liste_r = list(new_df[new_df['Concert Goer Age'].isna()]['Concert Goer ID'])
liste_r_index = new_df[new_df['Concert Goer Age'].isna()].index
for k in range (len(liste_r)):
    val = (new_df[new_df['Concert Goer ID']==liste_r[k]]['Concert Goer Age'].value_counts()).index[0]
    new_df.loc[liste_r_index[k], 'Concert Goer Age'] = val

In [61]:
results = list(new_df[colonnes[13]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[13], 'on a maintenant' , somme, 'entrées')

pour Concert Goer Age on a maintenant 200000 entrées


#### Colonne 14 : Concert Goer ID

In [62]:
results = list(new_df[colonnes[14]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[14], 'on a' , somme, 'entrées')

pour Concert Goer ID on a 200000 entrées


#### Colonne 15 : Height (cm)

In [63]:
results = list(new_df[colonnes[15]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[15], 'on a' , somme, 'entrées')

pour Height (cm) on a 199013 entrées


In [64]:
liste_r = list(new_df[new_df['Height (cm)'].isna()]['Concert Goer ID'])
liste_r_index = new_df[new_df['Height (cm)'].isna()].index
for k in range (len(liste_r)):
    val = (new_df[new_df['Concert Goer ID']==liste_r[k]]['Height (cm)'].value_counts()).index[0]
    new_df.loc[liste_r_index[k], 'Height (cm)'] = val

In [65]:
results = list(new_df[colonnes[15]].value_counts())
somme =0
for k in range(len (results)): 
    somme += results[k]
print('pour', colonnes[15], 'on a maintenant' , somme, 'entrées')

pour Height (cm) on a maintenant 200000 entrées


## Exportation

In [5]:
new_df.to_csv('prepro.csv')

NameError: name 'new_df' is not defined

In [9]:
new_df = pd.read_csv(r'prepro.csv')

In [11]:
new_df[new_df['Concert Attendance'] > 1000000]

,Unnamed: 0,Band Name,Band Genre,Band Country of Origin,Band Debut,Concert ID,Concert Attendance,Inside Venue,Rain,Seated,Personnality Trait 1,Personnality Trait 2,Personnality Trait 3,Personnality Trait 4,Concert Goer Age,Concert Goer ID,Height (cm),Concert Goer Country of Origin
3349,3349,19,6,144,1983.0,943.0,1.096000e+07,True,False,True,-0.330953,-0.035585,0.680300,0.644960,57.0,1722,173.0,135
5166,5166,24,2,144,1979.0,411.0,2.980000e+07,True,False,True,-0.095859,-0.787721,0.794274,0.583554,41.0,1294,172.0,144
5418,5418,51,5,144,1976.0,573.0,5.987400e+08,False,False,False,-0.362725,-0.504123,0.153142,0.821562,35.0,404,164.0,144
5653,5653,43,3,144,1976.0,658.0,5.987400e+08,True,False,True,0.040520,1.891511,0.160245,-0.805622,23.0,503,175.0,24
6248,6248,34,6,144,1980.0,427.0,4.030000e+06,False,False,True,0.668493,0.051597,-2.105785,0.388392,66.0,1127,180.0,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192399,22399,48,3,144,1979.0,317.0,8.103000e+07,True,False,True,-0.029212,-1.741027,0.788522,-0.201446,37.0,477,175.0,24
193971,23971,37,2,144,2000.0,23.0,4.030000e+06,True,False,False,-0.371054,-1.231813,-0.535272,2.036258,64.0,1504,158.0,24
194580,24580,34,6,144,1980.0,692.0,1.627540e+09,True,False,True,-0.133532,0.658807,0.887363,1.044550,25.0,815,158.0,47
194626,24626,29,5,144,1992.0,674.0,4.030000e+06,False,False,True,-0.200633,0.292829,-0.706950,2.170535,61.0,622,151.0,24
